## Note : 
Mean free path : $\lambda = \frac{1}{n \sigma}$

If $n \rightarrow 100 \times n$, then :
- $\lambda \rightarrow \frac{\lambda}{100}$
- $\text{cell size} \rightarrow \frac{\text{cell size}}{100}$ ($100^2$ more cells because the grid is 2D)
- $dt \rightarrow \frac{dt}{100}$

But you can lower, down to a certain extent, the number of particles per cell to "not incease too much the computation time".

## Representative physical length scale

The system has two lenghts : 
* lenght of the tube : $L = 0.01$ $m$
* width : $w = 0.001$ $m$

The width is the representative physical length scale that constrains the flow.

So : $K_d = \frac{\lambda}{w}$.




In [1]:
%matplotlib widget

# notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as scipy
from pathlib import Path

from lppydsmc.data.saver import Saver
from plotting import analysis
from lppydsmc.utils import physics

# plt.figure.max_open_warning = False

## Loading results

In [2]:
dir_path = Path('results/')
dz = 0.001

In [3]:
dt = 1e-7
mr = 1
name = 'neutralization.h5'
lenght = 0.011 # m
width = 0.005
cell_size = 0.001
temperature = 23e3 # K
nb_cells = 33
volume = cell_size*cell_size*dz*nb_cells


In [4]:
store = pd.HDFStore(dir_path/name)

In [5]:
store.keys()

['/collisions_with_walls', '/df', '/df_out_particles', '/total_deleted']

In [6]:
# defining each variable
df = store['df']
df_out_particles = store['df_out_particles']
total_deleted = store['total_deleted']
collisions_with_walls = store['collisions_with_walls']

In [8]:
choice = 'I'
gamma = 5/3. # roughly for atomes, for diatomic molecule : 7/5.


mass = physics.get_mass_part(53+1, 53, 74) # I
molecular_mass = 0.1269 # kg/mol

verbose=False

In [9]:
sound_vel = physics.speed_of_sound(molecular_mass, temperature, gamma)
print(f'Speed of sound for [{choice}] at temperature {temperature} K : {sound_vel} m/s')

Speed of sound for [I] at temperature 23000.0 K : 1584.7563179678798 m/s


## Note : 
To save a figure, you can use : `analysis.save_fig(fig, path, title = None, dpi = 400, figsize = None)`, which allows you to easily save a figure while giving it a title and changing its size.
You can also use the interactive widget.

## Choosing the part of the system to plot 

In [10]:

res_x_0, res_y_0 = 11, 5

total_nb_cells = res_x_0*res_y_0

In [11]:
fig, ax = plt.subplots(1, constrained_layout = True, figsize = (res_x_0, res_y_0));

fact = 10
volume_cell_plot = volume/(fact*fact*res_x_0*res_y_0)
duration = df.index.unique().values.shape[0]
analysis.hist2d(ax, df, bins = (fact*res_x_0,fact*res_y_0), weights = mr*np.ones(df.shape[0])/(duration*volume_cell_plot), stat = 'count'); # TODO: add cmap

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Choosing frames to plot

In [12]:
unique_index = df.index.unique().values
nb_save = unique_index.shape[0]
iterations = np.max(unique_index)
adding_period = unique_index[1]-unique_index[0] # adding period - required to
# generally speaking, you choose frames so you have the steady state
frames = unique_index[int(0.0*nb_save):nb_save] 

if(verbose):
    print(f'Available frames :  {unique_index}')
    print(f'Max iteration : {iterations}')
    print(f'Choosen frames (for plotting) : \n{frames}')

## Out particles analysis

In [16]:
df_out_particles.head()

,x,y,vx,vy,vz,type
37,0.016078,0.001549,4253.281957,-2125.455270,262.110609,1.0
40,0.016272,0.001086,3998.902894,-3062.369335,927.440667,1.0
42,0.016119,0.002669,3806.535264,426.745980,1645.013717,1.0
45,0.016239,0.002545,3603.506278,825.890848,-633.707650,1.0
47,0.016201,0.003349,3392.533582,1320.493493,-610.444198,1.0


In [31]:
df_out_particles.groupby('type').count() # all were converted.

,x,y,vx,vy,vz
type,,,,,
1.0,821,821,821,821,821


## Particles analysis
### Plotting number of particles evolution

In [13]:
nb_parts = df['x'].groupby(df.index).agg('count').values

In [15]:
fig, ax = plt.subplots()
ax.plot(df.index.unique()*dt, nb_parts)
analysis.set_axis(ax, x = 'time', y = 'quantity')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Plotting density evolution

In [13]:
fig, ax = plt.subplots()
ax.plot(df.index.unique()*dt, nb_parts*mr/volume)
analysis.set_axis(ax, x = 'time', y = 'density')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Density as a function of x

In [14]:
fig, ax = plt.subplots()
bins = 100
volume_cell_plot = volume/bins
duration = df.index.unique().values.shape[0]
weights = mr*np.ones(df.shape[0])/(duration*volume_cell_plot)
analysis.hist1d(ax, df, 'x', bins = bins, density = False, color = 'default', weights = weights, histtype = 'step')
analysis.set_axis(ax, x = 'x', y = 'density');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Pressure in mTorr

In [15]:
pressure_torr = physics.pressure_torr(nb_parts*mr, volume, temperature)

In [16]:
fig, ax = plt.subplots()
ax.plot(df.index.unique()*dt, pressure_torr*1e3)
analysis.set_axis(ax, x = 'time', y = None)
ax.set_ylabel('pressure (mTorr)');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Velocity distribution

In [17]:
fig, axes = analysis.velocity_distribution(df, frames, bins = 100, density = True, sharex = False, sharey = False);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

df_mean = df.groupby(df.index).mean()
## Temperature evolution

First - not by cell but for the whole system.

### Theory : 

The kinetic energy density $w$ is

$w = \frac{3}{2}p(\vec{r},t) + n(\vec{r},t)\frac{1}{2} m |\vec{u}(\vec{r},t)|^2$ 

With $\vec{u}(\vec{r},t) = ((<(v_x, v_y, v_z)>)(\vec{r},t))^T$ thus : $|\vec{u}(\vec{r},t)| = |<\vec{v}>|$.

Here the drift is not zero. So we have to remove it to have only $\frac{3}{2}p(\vec{r},t) = w - n(\vec{r},t)\frac{1}{2} m |\vec{u}(\vec{r},t)|^2$.

This yields : 

$\frac{3}{2}p(\vec{r},t) = n(\vec{r},t)  <\frac{1}{2}mv^2> - n(\vec{r},t)\frac{1}{2} m |\vec{u}(\vec{r},t)|^2$

Which yields : 

$$ \frac{3}{2}p(\vec{r},t) = \frac{m n(\vec{r},t)}{2}  (<v^2> - |<\vec{v}>|^2) $$

With $p = nkT$.

Intuition : total energy - energy contains in the drift.

In [18]:
df['v'] = np.sqrt(df['vx']*df['vx']+df['vy']*df['vy']+df['vz']*df['vz']) # df.apply(analysis.speed_norm, axis = 1)
df['v2'] = df['v']**2
df_mean = df.groupby(df.index).mean()
df_mean['v_drift'] = np.sqrt(df_mean['vx']*df_mean['vx']+df_mean['vy']*df_mean['vy']+df_mean['vz']*df_mean['vz'])

In [19]:
# maxwellian temperature 
temperature_mb = np.pi*mass*df_mean['v']**2/(8*physics.BOLTZMAN_CONSTANT)

In [20]:
df_mean['temperature'] =  mass/(3.*physics.BOLTZMAN_CONSTANT) * (df_mean['v2']-df_mean['v_drift']**2)
temperatures = df_mean['temperature']

In [21]:
fig, ax = plt.subplots(1,2)
ax[0].plot(unique_index*dt,temperatures)
ax[1].plot(unique_index*dt,temperature_mb)

analysis.set_axis(ax[0], x = 'time', y = 'temperature')
analysis.set_axis(ax[1], x = 'time', y = 'temperature')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
fig, ax = plt.subplots()
ax.plot(unique_index*dt,temperatures)
analysis.set_axis(ax, x = 'time', y = 'temperature')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
fig, ax = plt.subplots()
ax.plot(unique_index*dt, physics.pressure_torr(nb_parts*mr, volume, temperature_mb)*1e3)
analysis.set_axis(ax, x = 'iteration', y = None)
ax.set_ylabel('pressure (mTorr)');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
sound_speed = physics.speed_of_sound(molecular_mass, temperature_mb, gamma)

In [25]:
fig, ax = plt.subplots()
ax.plot(unique_index*dt,sound_speed)
analysis.set_axis(ax, x = 'time', y = 'sound speed')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Flow speed

In [26]:
fig, ax = plt.subplots()
ax.plot(unique_index*dt,df_mean['v_drift'])
analysis.set_axis(ax, x = 'time', y = 'v');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Mach number

In [27]:
fig, ax = plt.subplots()
ax.plot(unique_index*dt,df_mean['v_drift']/sound_speed)
analysis.set_axis(ax, x = 'time')
ax.set_ylabel('Mach number');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Collision with walls - analysis

In [28]:
fig, ax = plt.subplots()
analysis.set_axis(ax, x = 'time', y = 'quantity')
ax.plot(collisions_with_walls.index*dt, collisions_with_walls*mr);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Out particle analysis

In [29]:
fig, ax = plt.subplots()
mass_flow_rate = mr*physics.compute_mass_flow_rate(total_deleted, dt, mass)
analysis.set_axis(ax, x = 'time', y = 'mass flow rate')
plt.plot(mass_flow_rate.index*dt, mass_flow_rate);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
fig, ax = plt.subplots()
analysis.set_axis(ax, x = 'time', y = 'quantity')
plt.plot(total_deleted.index*dt, total_deleted);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Bonus - Not necessarily recommended and takes lots of time
Next is the plot of the system for each saved step. 

In [31]:
# creating speed_norm (for the color)
# df['speed_norm'] = np.sqrt(df['vx']**2+df['vy']**2+df['vz']**2) -> DO NOT NEED THAT

In [32]:
# %matplotlib notebook
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation

def update_hist(num, df, dt):
    if(num%1000==0):
        print('{}/{}'.format((num), iterations))
    
    dfit = df.loc[df.index == num]

    scat.set_offsets(np.c_[dfit['x'],dfit['y']])
    scat.set_array(df['v'])
    
    #ax.set_xlim((0.0,16e-3))
    #ax.set_ylim((0.0, 5e-3))
    ax.set_title('{:.3e}/{:.3e} s'.format(num*dt, iterations*dt), fontsize=15)

fig, ax = plt.subplots()
dfit = df.loc[df.index == 0]
scat = ax.scatter(dfit['x'], dfit['y'], s=0.1, c = dfit['v'], cmap='seismic') #  c = df['speed_norm']
analysis.set_axis(ax, x = 'x', y = 'y')
# if you want the boundaries to be plotted and the grid too, you have to initialize those fields yourself.
# in the other case, only particles will be plotted.
# plot_boundaries(ax, segments)
# plot_grid(ax, resolutions, system_shape)
ax.set_title('{}/{}'.format(1, iterations), fontsize=12)
#ax.axis('equal')

ax.set_xlim((0.0,16e-3))
ax.set_ylim((0.0, 5e-3))

interval = 10 # 160 images per second
anim = FuncAnimation(fig, update_hist, interval=interval, frames=frames[:5000], fargs=(df, dt), save_count=iterations)
plt.show()
# anim.save(dir_path/'system_state_evo.avi', dpi = 300);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …